<a href="https://colab.research.google.com/github/AlexeyUdod/netology_pyda/blob/master/Python_12_pivot_and_str2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [0]:
import pandas as pd

In [0]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [0]:
filtered = ratings[ (ratings['userId']== 1)]
filtered['rating'].value_counts().head()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
Name: rating, dtype: int64

In [0]:
ratings[ ratings.userId == 1 ].rating.value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [0]:
ratings[ratings.userId == 1].groupby('rating').count()[['userId']]

,userId
rating,
1.0,2
2.0,7
2.5,3
3.0,4
3.5,1
4.0,3


In [0]:
len(ratings.userId.unique())

671

Как это сделать для всех пользователей сразу:

In [0]:
?ratings.pivot_table

In [0]:
type(ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', 
                    aggfunc = 'count', fill_value=''))

pandas.core.frame.DataFrame

In [0]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', 
                    aggfunc = 'count', fill_value='').head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,,2,,7,3,4,1,3,,
2,,2,,4,,36,,23,,11
3,,,,1,3,18,9,11,4,5
4,,5,,5,,23,,52,,119
5,,,1,,3,3,27,42,19,5


In [0]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


In [0]:
# можно считать несколько функций сразу
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', 
                    aggfunc = ['count', 'sum']).tail()

count                                                 sum  \
rating   0.5  1.0 1.5  2.0  2.5   3.0   3.5   4.0  4.5   5.0 0.5   
userId                                                             
667      NaN  NaN NaN  4.0  NaN  28.0   NaN  24.0  NaN  12.0 NaN   
668      NaN  2.0 NaN  NaN  NaN   7.0   NaN   3.0  NaN   8.0 NaN   
669      NaN  NaN NaN  7.0  NaN  14.0   NaN  12.0  NaN   4.0 NaN   
670      NaN  2.0 NaN  3.0  NaN   5.0   NaN  10.0  NaN  11.0 NaN   
671      NaN  1.0 NaN  4.0  4.0   9.0  13.0  57.0  8.0  19.0 NaN   

                                                                    \
rating           1.0 1.5           2.0           2.5           3.0   
userId                                                               
667              NaN NaN  3.389087e+09           NaN  2.372360e+10   
668     1.987227e+09 NaN           NaN           NaN  6.955293e+09   
669              NaN NaN  7.110805e+09           NaN  1.422161e+10   
670     1.877564e+09 NaN  2.818508e+09           NaN  4.696072e+09   
671     1.063504e+09 NaN  4.259654e+09  4.265212e+09  9.579165e+09   

                                                                
rating           3.5           4.0           4.5           5.0  
userId                                                          
667              NaN  2.033452e+10           NaN  1.016726e+10  
668              NaN  2.980840e+09           NaN  7.948907e+09  
669              NaN  1.218995e+10           NaN  4.063317e+09  
670              NaN  9.387821e+09           NaN  1.033093e+10  
671     1.384568e+10  6.070199e+10  8.516026e+09  2.022592e+10

### Упражнение
Какой пользователь выставил больше всех пятерок?

In [0]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [0]:
ratings.pivot_table(index = ['userId', 'movieId'], columns = 'rating', values = 'timestamp', 
                    aggfunc = 'count', fill_value='').head()

rating         0.5 1.0 1.5 2.0 2.5 3.0 3.5 4.0 4.5 5.0
userId movieId                                        
1      31                        1                    
       1029                          1                
       1061                          1                
       1129                  1                        
       1172                                  1

In [0]:
sorted_tab = ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
sorted_tab.sort_values(by = 5, ascending=False).head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408
232,0,35,0,32,0,96,0,276,0,243
242,0,0,0,4,0,25,0,151,0,219
547,53,79,58,204,177,411,378,591,226,214
30,0,20,0,86,0,201,2,505,1,196


In [0]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).sort_values(by=5.0, ascending = False).head(1)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408


### Фильтры и вычисления с помощью loc и iloc

In [0]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [0]:
len(log.url.unique())

38

In [0]:
'https://host.ru/3c19b4ef7371864fa3'.split('/')[-1]

'3c19b4ef7371864fa3'

In [0]:
log.url.apply(lambda x: x.split('/')[-1]).head()

0    3c19b4ef7371864fa3
1    c8d9213a31839f9a3a
2    b8b58337d272ee7b15
3    b8b58337d272ee7b15
4    b8b58337d272ee7b15
Name: url, dtype: object

In [0]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [0]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

,visit_id,region
0,e3b0c44298,Russia
1,6e340b9cff,Russia
2,96a296d224,Russia
3,709e80c884,Russia
4,df3f619804,Russia


In [0]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [0]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 0.20
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,0.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,0.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,0.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,0.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,0.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,0.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,0.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,0.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,0.2


In [0]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [0]:
import pandas as pd


In [0]:
log = pd.read_csv('visit_log.csv', sep=';')
log.loc[(
            (log.traffic_source == 'yandex') | (log.traffic_source == 'google')
        ), 'source_type'] = 'organic'

log.loc[(
            ((log.traffic_source == 'paid')|(log.traffic_source == 'email'))
            & (log.region == 'Russia')
        ), 'source_type'] = 'ad'

log.loc[(
            ((log.traffic_source == 'paid')|(log.traffic_source == 'email'))
            & (log.region != 'Russia')
        ), 'source_type'] = 'other'

log.head(50)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


In [0]:
log = pd.read_csv('visit_log.csv', sep=';')
log.loc[(log.traffic_source == ('yandex' | 'google'), 'source_type')] = 'organic'
# log.loc[(
#     (log.traffic_source == 'paid')

# )]
log.head(50)

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [0]:

log.head(5)

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [0]:
log.loc[log['traffic_source'].str.contains('yandex|google'), 'traffic_type'] = 'organic'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [0]:
log.loc[log['traffic_source'].isin(['yandex', 'google']), 'traffic_type'] = 'organic'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [0]:
dict_ = {'a': 1}

In [0]:
print(dict_.get('b'))

None


In [0]:
import numpy as np

In [0]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [0]:
(123 + 456) / 2

289.5

In [0]:
df.value.mean()

289.5

In [0]:
pd.DataFrame({'value': [123, 0, 0, 0, 0, 456]}).mean()

value    96.5
dtype: float64

In [0]:
df[~pd.isnull(df.value)]

,value
0,123.0
5,456.0


In [0]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [0]:
log.loc[pd.isnull(log.traffic_type), 'traffic_type'] = 'other'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [0]:
log.loc[pd.isnull(log.traffic_type), 'traffic_type'] = 'other'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,other,other,other,other,other,other,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Дата и время в pandas

In [0]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [0]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 7 columns):
timestamp         18938 non-null int64
visit_id          18938 non-null object
url               18938 non-null object
region            18938 non-null object
user_id           18938 non-null object
traffic_source    18938 non-null object
traffic_type      18938 non-null object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


In [0]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,2019-02-12 14:12:16


In [0]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 8 columns):
timestamp         18938 non-null int64
visit_id          18938 non-null object
url               18938 non-null object
region            18938 non-null object
user_id           18938 non-null object
traffic_source    18938 non-null object
traffic_type      18938 non-null object
date              18938 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.2+ MB


In [0]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,2019-02-12 14:11:32,14
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other,2019-02-12 14:11:44,14
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,2019-02-12 14:11:55,14
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,2019-02-12 14:12:05,14
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,2019-02-12 14:12:16,14


### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [0]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

,visit_datetime,purchase_datetime
0,2019-11-04T00:05:13,2019-11-04T00:15:53
1,2019-11-04T00:06:19,2019-11-04T00:07:59
2,2019-11-04T01:35:14,2019-11-04T03:15:30


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
visit_datetime       3 non-null object
purchase_datetime    3 non-null object
dtypes: object(2)
memory usage: 176.0+ bytes


In [0]:
df['visit_dt'] = pd.to_datetime()

In [0]:
df['visit_dt'] = pd.to_datetime(df['visit_datetime'])
df['purchase_dit'] = pd.to_datetime(df['purchase_datetime'])
df['Delay'] = (df['purchase_dit'] - df['visit_dt']).astype('timedelta64[m]')
df['Delay'].mean()

37.0

In [0]:
df['visit_dt']=pd.to_datetime(df['visit_datetime'], format='%Y-%m-%dT')
df['purchase_dt']=pd.to_datetime(df['purchase_datetime'])
df['delta_dt'] = (df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
df.delta_dt.mean()

37.0

In [0]:
df.head()

,visit_datetime,purchase_datetime,visit_dt,purchase_dt,delta_dt,purchase_dit,Delay
0,2019-11-04T00:05:13,2019-11-04T00:15:53,2019-11-04 00:05:13,2019-11-04 00:15:53,10.0,2019-11-04 00:15:53,10.0
1,2019-11-04T00:06:19,2019-11-04T00:07:59,2019-11-04 00:06:19,2019-11-04 00:07:59,1.0,2019-11-04 00:07:59,1.0
2,2019-11-04T01:35:14,2019-11-04T03:15:30,2019-11-04 01:35:14,2019-11-04 03:15:30,100.0,2019-11-04 03:15:30,100.0


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
visit_datetime       3 non-null object
purchase_datetime    3 non-null object
visit_dt             3 non-null datetime64[ns]
purchase_dt          3 non-null datetime64[ns]
delta_dt             3 non-null float64
purchase_dit         3 non-null datetime64[ns]
Delay                3 non-null float64
dtypes: datetime64[ns](3), float64(2), object(2)
memory usage: 296.0+ bytes


### Методы работы со строками

In [0]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [0]:
stats[stats.keyword.str.contains('москва')].head()

,keyword,shows
127,авито москва,979292
1259,москва,153050
1396,гисметео москва,140279
1482,госуслуги москва,133061
1682,погода москва,151701


Проверка наличия подстроки в строке в питоне:

In [0]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [0]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [0]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [0]:
stats[stats.keyword.str.contains('погода в|Москва|Новосибирск|Краснодар')].head()

,keyword,shows
197,погода в москве,745745
414,погода в москве на 14 дней,400914
564,погода в санкт петербурге,292489
651,погода в краснодаре,261285
733,погода в самаре,234646


In [0]:
stats[stats.keyword.str.contains('(погода в){1}(|Москва|Новосибирск|Краснодар)')].head()

/Users/kbashevoy/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,keyword,shows
197,погода в москве,745745
414,погода в москве на 14 дней,400914
564,погода в санкт петербурге,292489
651,погода в краснодаре,261285
733,погода в самаре,234646


In [0]:
stats[stats.keyword.str.contains('погода в'and'москве|новосибирске|краснодаре')].head()

,keyword,shows
197,погода в москве,745745
414,погода в москве на 14 дней,400914
651,погода в краснодаре,261285
652,работа в москве,265000
1287,авто ру авто с пробегом в москве,153900


In [0]:
stats[
    (stats.keyword.str.contains('погода в')) & 
    (stats.keyword.str.contains('москв|новосибирск|краснодар'))
].head(20)

,keyword,shows
197,погода в москве,745745
414,погода в москве на 14 дней,400914
651,погода в краснодаре,261285
1342,погода в москве на неделю,180096
1498,погода в новосибирске,133362
1867,погода в москве на 3 дня,115394
2316,погода в новосибирске на 14 дней,93278
4930,погода в краснодаре на 10 дней,49354
6483,погода в москве на 14,40531
8645,погода в москве на месяц,43801


### replace

In [0]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

'отпуск начнется через месяц'

Аналог в pandas на запросах про сериалы:

In [0]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

,keyword,shows
246,сериалы,587779
304,сериалы тут,503967
555,турецкие сериалы на русском языке,296403
881,русские сериалы,234262
890,сериалы онлайн,204812


In [0]:
serial.keyword.str.replace('сериалы', 'книги').head()

246                              книги
304                          книги тут
555    турецкие книги на русском языке
881                      русские книги
890                       книги онлайн
Name: keyword, dtype: object

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [0]:
serial.keyword.str.upper().head()

246                              СЕРИАЛЫ
304                          СЕРИАЛЫ ТУТ
555    ТУРЕЦКИЕ СЕРИАЛЫ НА РУССКОМ ЯЗЫКЕ
881                      РУССКИЕ СЕРИАЛЫ
890                       СЕРИАЛЫ ОНЛАЙН
Name: keyword, dtype: object

In [0]:
serial.keyword.str.lower().head()

246                              сериалы
304                          сериалы тут
555    турецкие сериалы на русском языке
881                      русские сериалы
890                       сериалы онлайн
Name: keyword, dtype: object

### Что делать если нужно учесть формы написания слов?

In [0]:
stats[stats.keyword.str.contains('рубл')].head()

,keyword,shows
1202,алиэкспресс на русском в рублях официальный сайт,161553
1602,курс гривны к рублю,125076
1736,али экспресс русская версия на русском в рубля...,117260
2132,доллары в рубли,114173
2172,100 долларов в рублях,97534


Почему нельзя просто оставить str.contains('рубл'):

In [0]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

Еще популярный вариант - NLTK

In [0]:
from pymystem3 import Mystem

In [0]:
search = 'курс гривны к рублю рубли рублях рубленая котлетка однушками рублишка культяпками'

In [0]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубленый',
 ' ',
 'котлетка',
 ' ',
 'однушка',
 ' ',
 'рублишко',
 ' ',
 'культяпка',
 '\n']

In [0]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль   рубленый   котлетка   однушка   рублишко   культяпка \n'

In [0]:
!/Users/kbashevoy/anaconda3/bin/pip install pymystem3

In [0]:
m.analyze(search)[2]

{'analysis': [{'lex': 'гривна',
   'wt': 1,
   'gr': 'S,жен,неод=(вин,мн|род,ед|им,мн)'}],
 'text': 'гривны'}

In [0]:
%%time

for _ in range(10):
    m.analyze('стали')

CPU times: user 1 ms, sys: 1.37 ms, total: 2.37 ms
Wall time: 6.8 ms


### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [0]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [0]:
data